In [1]:
import pandas as pd
import numpy as np
import gc
from joblib import dump, load
from datetime import datetime, date

from rec_lib.utils import reduce_mem_usage, create_transactions_dict

import warnings
warnings.filterwarnings("ignore")

In [2]:
# загружаем данные
df_articles = pd.read_csv('archive/articles.csv')
df_customers = pd.read_csv('archive/customers.csv')

In [3]:
# сохраняем в формате parquet
df_articles.to_parquet('archive/articles.parquet')
df_customers.to_parquet('archive/customers.parquet')

In [4]:
# функция вычисления порядкового номера недели покупки
def week_num(el):   
    global spam
    global spam2
    
    if spam != el:
        spam = el
        spam2 += 1
        
    return spam2

In [5]:
# функция создания датафрейма, сгруппированного по количеству покупок категории покупателем, с дополнительными колонками недель
def df_transactions_prepare(transactions_dict):
    
    global spam
    
    df_transactions = pd.DataFrame([(k+','+str(v)).split(',') for k,v in transactions_dict.items()], columns=['t_dat','customer_id','article_id','price','sales_channel_id', 'values'])
    
    # преобразуем форматы данных
    df_transactions['t_dat'] = df_transactions['t_dat'].astype('datetime64[ns]')
    df_transactions['article_id'] = df_transactions['article_id'].astype('int32')
    df_transactions['sales_channel_id'] = df_transactions['sales_channel_id'].astype('int32')
    df_transactions['values'] = df_transactions['values'].astype('int32')
    df_transactions['price'] = df_transactions['price'].astype('float')
    
    # неделя года на которой совершена покупку
    df_transactions['week_number_of_year'] = df_transactions['t_dat'].dt.week
    df_transactions['week_number_of_year'] = df_transactions['week_number_of_year'].astype('int32')
    
    # для анализа по неделям уберем начальные дни из данных для того, чтобы в неделях было одинаковое количество дней
    df_transactions = df_transactions.loc[~df_transactions['t_dat'].isin(['2018-09-20 00:00:00', '2018-09-21 00:00:00', '2018-09-22 00:00:00', '2018-09-23 00:00:00', '2018-09-24 00:00:00', '2018-09-25 00:00:00'])]
    
    # добавляем порядковый номер недели на которой совершена покупка
    spam = df_transactions.iloc[0]['week_number_of_year'] # номер недели года
    week_number_of_year = np.array(df_transactions['week_number_of_year'])
    week_num_arr = [week_num(x) for x in week_number_of_year]
    df_transactions['week_number'] = week_num_arr
    df_transactions['week_number'] = df_transactions['week_number'].astype('int32')
    
    # создаем справочники users и items для более быстрой работы кода
    customer_id_short = np.arange(0,df_customers.shape[0])
    userid_to_id = dict(zip(df_customers.customer_id, customer_id_short))
    id_to_userid= dict(zip(customer_id_short, df_customers.customer_id))

    article_id_short = np.arange(0,df_articles.shape[0])
    itemid_to_id = dict(zip(df_articles.article_id, article_id_short))
    id_to_itemid = dict(zip(article_id_short, df_articles.article_id))
    
    # добавляем в df_transactions столбцы сокращенных id для users и items
    df_transactions_customer_id = list(df_transactions.customer_id)
    df_transactions_customer_id_short = np.array([userid_to_id[el] for el in list(df_transactions.customer_id)])

    df_transactions_article_id = list(df_transactions.article_id)
    df_transactions_article_id_short = np.array([itemid_to_id[el] for el in list(df_transactions.article_id)])
    
    df_transactions['customer_id_short'] = df_transactions_customer_id_short
    df_transactions['article_id_short'] = df_transactions_article_id_short
    df_transactions['customer_id_short'] = df_transactions['customer_id_short'].astype('int32')
    df_transactions['article_id_short'] = df_transactions['article_id_short'].astype('int32')
    
    return df_transactions

In [6]:
%%time
# создаем словарь количества покупок пользователем одной категории товара за транзакцию (избавляемся от дублей)
transactions_dict = create_transactions_dict()

CPU times: user 21.5 s, sys: 6.35 s, total: 27.9 s
Wall time: 27.9 s


In [7]:
%%time
# создаем датафрейм, сгруппированный по количеству покупок категории покупателем, с дополнительными колонками недель
spam = 1 # исходный номер недели года
spam2 = 1 # исходный номер начальной недели
df_transactions = df_transactions_prepare(transactions_dict)

CPU times: user 2min 11s, sys: 14.2 s, total: 2min 25s
Wall time: 2min 25s


In [8]:
# сокращаем объем занимаемой памяти df_articles
df_articles = reduce_mem_usage(df_articles)

Memory usage of dataframe is 20.13 MB
Memory usage after optimization is: 7.25 MB
Decreased by 64.0%


In [9]:
# сокращаем объем занимаемой памяти df_transactions
df_transactions = reduce_mem_usage(df_transactions)

Memory usage of dataframe is 1635.52 MB
Memory usage after optimization is: 1160.21 MB
Decreased by 29.1%


In [11]:
# сохраняем а формате parquet
df_transactions.to_parquet('archive/transactions_train_for_power_bi.parquet')